Importing the necessary libraries

In [2]:
import os
from loaders import parse_document

from pydantic import BaseModel
from typing import Literal, Sequence, Union, Annotated, List, TypedDict
from typing_extensions import TypedDict

from langchain_openai import ChatOpenAI
from langchain.schema import BaseMessage
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import START,END,StateGraph,MessagesState
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

OpenAI Model

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", streaming=True)

Agent State - messages gets appended by each agent

In [4]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    sender: str
    count: int
    user_feedback: str

In [5]:
#Reader Agent
def reader_call(state:AgentState):
    name="reader"
    reader_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an resume reading and preparing assistant. You will be given the contents of the resume that is parsed with a particular file reader.\n
         You are expected to prepare the resume data for processing further. The document should contain its originality and you should only beautify it\n
         Also you are not expected to provide any headers like 'Here is the resume' or any footers like 'This resume format maintains the originality of the content'. Instead just provide only the content.\n
         Remember that you are a part of a team. So you are expected to present file contents properly without any changes that affects the context."""),
        ("human", "{input}")
        ])
    
    reader_chain = reader_prompt | llm

    response = reader_chain.invoke({"input":state['messages']})
    return {"messages": [response.content],
            "sender": name,
            "count": state.get("count", 0)
    }

In [6]:
#Extractor Agent
def extractor_call(state:AgentState):
    name="extractor"
    extractor_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an resume parsing assistant. You will be given contents of the resume. You are expected to extract various information such as \n1. Personal Information \n2.Education \n3.Work Experience \n4.Skills.\
        Compile the validated entities into a predefined JSON format for downstream use. \n
        Remember that you are a part of a team. So you are expected to read the file contents properly and you expected to give the data properly.\
        If the extraction is not up to the mark, you will be reprompted to extract that particular values again."""),
        ("human", "{input}")
        ])
    
    extractor_chain = extractor_prompt | llm

    response = extractor_chain.invoke({"input":state['messages']})
    
    new_count = state.get("count", 0) + 1
    
    return {"messages": [response.content], "sender": name, "count": new_count}

In [7]:
#Validation Agent
def validator_call(state:AgentState):
    name="validator"
    validator_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a parsed resume validation assistant. You will be given the extracted entities from the resume.\n
                 Perform quality checks and ensure that the fields match with the actual content.
                 So provide corrections that the other agent will then look at and then extract them correctly.\n
         If there are quality errors, then provide them as a set of bullet points.\n
                Remember that you are a part of a team. So you are expected to present the corrections properly without any unwanted changes.\n
                 If you don't notice any changes or mistakes then just return 'Yes'. Also you should not provide any headers or additional information like 'There is no mistake', 'The extraction is correct', etc. Just provide the final 'Yes'."""),
        ("human", "{input}"),
        ])
    
    validator_chain = validator_prompt | llm


    response = validator_chain.invoke({"input":state['messages']})
    
    return {"messages": [response.content],
            "sender": name,
            "count":state.get("count",0)}

In [8]:
#Routing conditions
def router(state: AgentState):
    if state.get("count", 0) >= 3 or state["messages"][-1].content == 'Yes':
        return END
    elif state["sender"] == "reader":
        return "extractor"
    elif state["sender"] == "extractor":
        return "validator"
    elif state["sender"] == "validator":
        return "extractor"
    else:
        return "reader"

In [9]:
workflow = StateGraph(AgentState)

workflow.add_node("reader", reader_call)
workflow.add_node("extractor", extractor_call)
workflow.add_node("validator", validator_call)

workflow.add_conditional_edges(
    "reader",
    router,
    {
        "extractor": "extractor",
        END: END
    }
)

workflow.add_conditional_edges(
    "extractor",
    router,
    {
        "validator": "validator",
        END: END
    }
)

workflow.add_conditional_edges(
    "validator",
    router,
    {
        "extractor": "extractor",
        END: END
    }
)

workflow.add_edge(START, "reader")

In [10]:
memory = MemorySaver()
app = workflow.compile(checkpointer=memory,interrupt_before=['extractor'])

In [11]:
#Resume data to check workflow
data=""
documents=parse_document("Documents/yuans-resume-template.pdf")
for i in documents:
    data+=i.page_content

In [12]:
config = {"configurable": {"thread_id": "a8"}}
initial_state = {
    "messages": [HumanMessage(content=data)], #data-resume contents
    "count": 0,
    "sender": "",
}

Human input 

In [13]:
for event in app.stream(initial_state, config, stream_mode="values"):
    print(event)

try:
    user_approval = input("Do you want to continue the extraction : ")
except:
    user_approval = "yes"

if user_approval.lower() == "yes":
    # If approved, continue the graph execution
    for event in app.stream(None, config, stream_mode="values"):
        print(event)
else:
    print("Operation cancelled by user.")

{'messages': [HumanMessage(content='(+00) 111-2222-3333\nyuanhf@example.com\nhttp://www.example.com\nXIAO YUAN\nPH.D.\nEducation\nDepartment of Automation, Tsinghua University\nBeijing, China\nPh.D. in Control Science and Engineering\n2022 - 2028 (expected)\n• Advisor: Prof. Xiao Yuan\n• Research area: Operations Research and Machine Learning\nDepartment of Precision Instrument, Tsinghua University\nBeijing, China\nB.E. in Measurement and Control Technology and Instrument\n2018 - 2022\n• GPA: 0.00/4.00, Rank: 64/64.\nPublications\n1. Xiao Yuan, Hua Li. The Future Urban Transportation Systems: Innovations and Chal-\nlenges. Journal of Operations Research and Optimization, 2024.\n2. Hua Li, Xiao Yuan, John Doe. Optimizing Logistics and Supply Chain Networks Using\nMachine Learning Techniques. International Conference on Operations Research and\nMachine Learning, 2023.\n3. John Doe, Xiao Yuan, Hua Li. Artificial Intelligence in Healthcare: Transforming Diag-\nnostics and Treatment. Intern